In [4]:
#import libraries
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import sklearn

In [10]:
data_dir = '/Users/jyontika/Desktop/Python/GitHub/opioid-overdose-models/jyontika-MA-data/data/'

annual_path = os.path.join(data_dir, 'clean_annual_tract')
#quarter_path = os.path.join(data_dir, 'clean_annual_tract.shp')
#semi_path = os.path.join(data_dir, 'clean_annual_tract.shp')

gdf_annual = gpd.read_file(annual_path)
#gdf_quarter = gpd.read_file(quarter_path)
#gdf_semi = gpd.read_file(semi_path)

In [11]:
gdf_annual

,geoid,year,deaths,month,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,...,month_sinc,season,season_sin,quarter,qtr_since_,year_since,self_t-1,neighbor_t,timestep,geometry
0,25001010100,2000,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,0.0,jan-jun,0.0,1.0,0.0,0.0,0.0,0.000,0.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
1,25001010100,2001,5.0,1,25,001,10100,101,Census Tract 101,G5020,...,12.0,jan-jun,2.0,1.0,4.0,1.0,0.0,0.000,1.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
2,25001010100,2002,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,24.0,jan-jun,4.0,1.0,8.0,2.0,0.0,0.000,2.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
3,25001010100,2003,1.0,1,25,001,10100,101,Census Tract 101,G5020,...,36.0,jan-jun,6.0,1.0,12.0,3.0,0.0,0.000,3.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
4,25001010100,2004,0.0,1,25,001,10100,101,Census Tract 101,G5020,...,48.0,jan-jun,8.0,1.0,16.0,4.0,0.0,0.000,4.0,"POLYGON ((-70.25001 42.06410, -70.24959 42.065..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35635,25027761402,2017,0.0,1,25,027,761402,7614.02,Census Tract 7614.02,G5020,...,204.0,jan-jun,34.0,1.0,68.0,17.0,0.0,0.125,17.0,"POLYGON ((-71.63921 42.53096, -71.63906 42.531..."
35636,25027761402,2018,1.0,1,25,027,761402,7614.02,Census Tract 7614.02,G5020,...,216.0,jan-jun,36.0,1.0,72.0,18.0,0.0,0.125,18.0,"POLYGON ((-71.63921 42.53096, -71.63906 42.531..."
35637,25027761402,2019,0.0,1,25,027,761402,7614.02,Census Tract 7614.02,G5020,...,228.0,jan-jun,38.0,1.0,76.0,19.0,0.0,0.250,19.0,"POLYGON ((-71.63921 42.53096, -71.63906 42.531..."
35638,25027761402,2020,0.0,1,25,027,761402,7614.02,Census Tract 7614.02,G5020,...,240.0,jan-jun,40.0,1.0,80.0,20.0,0.0,0.000,20.0,"POLYGON ((-71.63921 42.53096, -71.63906 42.531..."


In [14]:
annual_deaths = gdf_annual.groupby('year')['deaths'].sum().reset_index()

# Calculate sparsity rate
zero_deaths_count = gdf_annual[gdf_annual['deaths'] == 0].groupby('year').size().reset_index(name='Zero Deaths Count')
total_tracts_count = gdf_annual.groupby('year').size().reset_index(name='Total Tracts Count')
sparsity_df_annual = pd.merge(zero_deaths_count, total_tracts_count, on='year')
sparsity_df_annual['Sparsity Rate'] = (sparsity_df_annual['Zero Deaths Count'] / sparsity_df_annual['Total Tracts Count']) * 100
sparsity_df_annual['Sparsity Rate'] = sparsity_df_annual['Sparsity Rate'].round(3).astype(str) + '%'

# Merge sparsity rate with annual_deaths table
annual_deaths = pd.merge(annual_deaths, sparsity_df_annual[['year', 'Sparsity Rate']], on='year')

# Rename the 'deaths' column
annual_deaths.rename(columns={'deaths': 'Total Deaths'}, inplace=True)

# Exclude 2023 from the table
annual_deaths = annual_deaths[annual_deaths['year'] != 2023]

# Print the resulting table
print(annual_deaths)


    year  Total Deaths Sparsity Rate
0   2000         355.0       81.173%
1   2001         445.0       77.531%
2   2002         476.0       76.605%
3   2003         596.0       71.975%
4   2004         469.0       76.358%
5   2005         547.0       74.259%
6   2006         613.0       70.247%
7   2007         599.0       71.358%
8   2008         597.0       70.988%
9   2009         595.0       71.975%
10  2010         524.0       73.951%
11  2011         621.0       70.123%
12  2012         703.0       67.346%
13  2013         897.0       61.667%
14  2014        1247.0       52.346%
15  2015        1575.0       46.605%
16  2016        1896.0       39.383%
17  2017        1711.0       42.963%
18  2018        1895.0       40.432%
19  2019        1862.0       40.185%
20  2020        1953.0       39.815%
21  2021        2064.0       39.383%


In [16]:
#number of census tracts with 5+ deaths in a year
result = gdf_annual.groupby('year').apply(lambda x: (x['deaths'] >= 5).sum()).reset_index()

result.columns = ['Year', '   # of Tracts']
print("Number of Census Tracts with\n  5+ Deaths in Given Year\n")
print(result)


Number of Census Tracts with
  5+ Deaths in Given Year

    Year     # of Tracts
0   2000               0
1   2001               1
2   2002               0
3   2003               0
4   2004               1
5   2005               3
6   2006               0
7   2007               0
8   2008               0
9   2009               0
10  2010               3
11  2011               3
12  2012               3
13  2013               4
14  2014              15
15  2015              31
16  2016              44
17  2017              38
18  2018              51
19  2019              40
20  2020              59
21  2021              66
